# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [9]:
base = pd.read_csv("Cientista de dados M35 - smoke_detection_iot.csv", delimiter=',')

In [10]:
base.drop(columns=['Unnamed: 0'], inplace=True)
base.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)
base

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire_Alarm
0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,0
1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,0
2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,0
3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,0
4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,1655130047,18.438,15.79,625,400,13723,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,0
62626,1655130048,18.653,15.87,612,400,13731,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,0
62627,1655130049,18.867,15.84,627,400,13725,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,0
62628,1655130050,19.083,16.04,638,400,13712,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,0


In [7]:
base.dtypes

UTC                 int64
Temperature[C]    float64
Humidity[%]       float64
TVOC[ppb]           int64
eCO2[ppm]           int64
Raw H2              int64
Raw Ethanol         int64
Pressure[hPa]     float64
PM1.0             float64
PM2.5             float64
NC0.5             float64
NC1.0             float64
NC2.5             float64
CNT                 int64
Fire Alarm          int64
dtype: object

In [8]:
base.isnull().sum()

UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64

Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.




Usarei a regressão logística porque é um modelo mais simples, mais interpretável e com menor risco de overfitting em comparação com modelos mais complexos como a árvore de decisão.

# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [11]:
X = base.drop('Fire_Alarm', axis=1)
y = base['Fire_Alarm']

In [12]:
# Instanciando o modelo de regressão logística
modelo = LogisticRegression(max_iter=1000)

# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [13]:
folds = 5

In [14]:
crossvalidation = KFold(n_splits=folds, shuffle=True, random_state=5)

In [15]:
modelo_final = cross_val_score(modelo, X, y, cv = folds)

# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [18]:
pontuacoes = cross_val_score(modelo, X, y, cv=crossvalidation)

print("Acurácia por fold: ", [f"{p:.2f}" for p in pontuacoes])

Acurácia por fold:  ['0.98', '0.98', '0.98', '0.98', '0.98']


In [19]:
print(f"Acurácia média: {pontuacoes.mean():.2f}")

Acurácia média: 0.98
